In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys

if "../" not in sys.path:
    sys.path.append("../")
    print("[sys.path]:", sys.path)

[sys.path]: ['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/nadir/MoLiNER/.venv/lib/python3.10/site-packages', '../']


In [1]:
import os
import torch
import pytest

In [ ]:
from src.types import ForwardOutput, ProcessedBatch
from src.model.modules.losses.helpers.target_matrix import create_target_matrix, create_target_matrix_iou

/home/nadir/MoLiNER/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
BATCH_SIZE = 1
NUMBER_OF_CANDIDATE_SPANS = 3
NUMBER_OF_GROUNDTRUTH_SPANS = 1

# NOTE: (batch_size, batch_max_spans_per_motion_in_batch, 2) -> [start_frame, end_frame]
CANDIDATE_SPANS = torch.tensor([
    [
        # [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0667, 0.1333, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0714, 0.1429, 0.2143, 0.2857]
        [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8],
        [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9],
    ]
])

# NOTE: (batch_size, batch_max_prompts_per_motion, batch_max_spans_per_prompt, 2)
TARGET_SPANS = torch.tensor([
    [
        # Prompt 1 Spans
        [[0, 6], [1, 9], [5, 9], [8, 10]],
        # Prompt 2 Spans
        [[5, 15], [10, 20], [15, 25], [20, 30]]
    ]
])

NUMBER_OF_PROMPTS = TARGET_SPANS.shape[1]
NUMBER_OF_CANDIDATE_SPANS = CANDIDATE_SPANS.shape[1]
NUMBER_OF_GROUNDTRUTH_SPANS = TARGET_SPANS.shape[2]

print(f"[BATCH_SIZE]: {BATCH_SIZE}")
print(f"[NUMBER_OF_PROMPTS]: {NUMBER_OF_PROMPTS}")
print(f"[NUMBER_OF_CANDIDATE_SPANS]: {NUMBER_OF_CANDIDATE_SPANS}")
print(f"[NUMBER_OF_GROUNDTRUTH_SPANS]: {NUMBER_OF_GROUNDTRUTH_SPANS}")

TARGET_SPANS_PER_PROMPT_MASK = torch.tensor([
    [
        # Prompt 1
        [True, True, True, True],
        # Prompt 2
        [True, False, False, False],
    ]
])

SIMILARITY_MATRIX = torch.randn(BATCH_SIZE, NUMBER_OF_PROMPTS, NUMBER_OF_CANDIDATE_SPANS)
CANDIDATE_SPANS_MASK = torch.ones(BATCH_SIZE, NUMBER_OF_CANDIDATE_SPANS)
BATCH = ProcessedBatch(
    sid=[1],
    dataset_name=["test"],
    amass_relative_path=["test.npz"],
    motion_features=torch.randn(BATCH_SIZE, 100, 263),
    motion_mask=torch.ones(BATCH_SIZE, 100),
    prompt_input_ids=torch.zeros(BATCH_SIZE, NUMBER_OF_PROMPTS, 10, dtype=torch.long),
    prompt_attention_mask=torch.ones(BATCH_SIZE, NUMBER_OF_PROMPTS, 10),
    target_spans=TARGET_SPANS,
    target_spans_mask=torch.ones(BATCH_SIZE, NUMBER_OF_PROMPTS),
    target_spans_per_prompt_mask=TARGET_SPANS_PER_PROMPT_MASK,
    is_sequence_prompt=torch.tensor([[True]])
)
FORWARD_OUTPUT = ForwardOutput(
    similarity_matrix=SIMILARITY_MATRIX,
    candidate_spans_indices=CANDIDATE_SPANS,
    candidate_spans_mask=CANDIDATE_SPANS_MASK,
    prompts_mask=torch.ones(BATCH_SIZE, NUMBER_OF_PROMPTS)
)

[BATCH_SIZE]: 1
[NUMBER_OF_PROMPTS]: 2
[NUMBER_OF_CANDIDATE_SPANS]: 16
[NUMBER_OF_GROUNDTRUTH_SPANS]: 4


In [86]:
print(f"[similarity_matrix]({SIMILARITY_MATRIX.shape}):")
print(SIMILARITY_MATRIX)

[similarity_matrix](torch.Size([1, 2, 16])):
tensor([[[-0.5770, -0.5185,  0.0646,  0.6118,  0.0308, -0.9128, -0.2207, -0.2385,  0.1002,  0.7006,  1.5171,  0.5459,
           0.1438, -0.1433,  0.4278, -0.9865],
         [ 0.3034, -2.3233,  0.2672,  0.8642, -0.1742, -0.9976, -0.7267,  0.6877, -0.1816, -1.4061,  1.6194, -0.7558,
           0.0813,  0.8257,  1.1304, -1.3611]]])


In [ ]:
iou_target_matrix, _ = create_target_matrix_iou(FORWARD_OUTPUT, BATCH, iou_threshold=.0)
target_matrix, _ = create_target_matrix(FORWARD_OUTPUT, BATCH, 1.0)

torch.set_printoptions(precision=4, sci_mode=False, linewidth=120)

print(f"[iou_target_matrix]({iou_target_matrix.shape}):")
print(iou_target_matrix)

print()

print(f"[target_matrix]({target_matrix.shape}):")
print(target_matrix)

[iou_target_matrix](torch.Size([1, 2, 16])):
tensor([[[0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000, 0.8571, 0.7778, 0.1667, 0.3333, 0.5000, 0.6667, 0.8333,
          0.7500, 0.8750, 1.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0667, 0.1333, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0714,
          0.1429, 0.2143, 0.2857]]])

[target_matrix](torch.Size([1, 2, 16])):
tensor([[[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])
